In [1]:
from google.colab import files
uploaded = files.upload()


Saving ForUsAll_External Technical Test CSV File - Employee Census Data.csv to ForUsAll_External Technical Test CSV File - Employee Census Data.csv


In [21]:
import pandas as pd
file_name = 'ForUsAll_External Technical Test CSV File - Employee Census Data.csv'

#In this part we are going to check to see if the file was uploaded successfully, otherwise an exception will be made.
try:
  df = pd.read_csv(file_name, na_values=['0', '0.00', '$0.00'],encoding='utf-8', thousands=',')
  print(f"Successfully uploaded {file_name}!")
  print(f"\n First 5 rows of the dataset:")
  print(df.head())
  print(f"\n Dataframe Info:")
  print(df.info())
except Exception as e:
  print(f"Error uploading {file_name} was not found. Please ensure it's uploaded correctly and the name matches. Error: {e}")



Successfully uploaded ForUsAll_External Technical Test CSV File - Employee Census Data.csv!

 First 5 rows of the dataset:
           ID Birth_Date_(MM/DD/YYYY) Hire_Date Termination_Date  Rehire_Date  \
0  Employee 1                 5/14/44    1/7/20              NaN          NaN   
1  Employee 2                12/18/31   1/19/19              NaN          NaN   
2  Employee 3                 8/28/48    1/1/20              NaN          NaN   
3  Employee 4                10/20/72  11/18/13              NaN          NaN   
4  Employee 5                  4/3/91    6/7/17          2/20/23          NaN   

  Gross_compensation Plan_eligible_comp Employee_deduction_1_Pre-tax_amount  \
0         $82,460.35         $82,460.35                          $17,219.53   
1         $36,328.55         $36,328.55                           $1,018.17   
2        $319,773.45        $319,773.45                           $8,716.71   
3         $60,120.66         $60,120.66                                 Na

In [24]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print(df.columns)





Index(['ID', 'Birth_Date_MM/DD/YYYY', 'Hire_Date', 'Termination_Date',
       'Rehire_Date', 'Gross_compensation', 'Plan_eligible_comp',
       'Employee_deduction_1_Pre-tax_amount',
       'Employee_deduction_2_Roth_Deferral', 'Employer_Match',
       'Hours_worked_2022', 'Employee_Status', 'Entry_date'],
      dtype='object')


In [28]:
date_columns = ['Birth_Date_MM/DD/YYYY', 'Hire_Date', 'Termination_Date', 'Rehire_Date', 'Entry_date']
clean_date_columns = [col.strip().replace(' ', '_').replace('(', '').replace(')', '') for col in date_columns]
for col in clean_date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce', format='%m/%d/%Y')

df['Birth_Date_MM/DD/YYYY'] = df['Birth_Date_MM/DD/YYYY'].apply(lambda x: x - pd.DateOffset(years=100) if pd.notnull(x) and x.year > current_year - 18 else x)

print(df[clean_date_columns].head())

  Birth_Date_MM/DD/YYYY  Hire_Date Termination_Date Rehire_Date Entry_date
0            1944-05-14 2020-01-07              NaT         NaT 2020-03-01
1            1931-12-18 2019-01-19              NaT         NaT 2019-03-01
2            1948-08-28 2020-01-01              NaT         NaT 2020-02-01
3            1972-10-20 2013-11-18              NaT         NaT 2014-01-01
4            1991-04-03 2017-06-07       2023-02-20         NaT 2017-08-01


In [30]:
print(df.head())

           ID Birth_Date_MM/DD/YYYY  Hire_Date Termination_Date Rehire_Date  \
0  Employee 1            1944-05-14 2020-01-07              NaT         NaT   
1  Employee 2            1931-12-18 2019-01-19              NaT         NaT   
2  Employee 3            1948-08-28 2020-01-01              NaT         NaT   
3  Employee 4            1972-10-20 2013-11-18              NaT         NaT   
4  Employee 5            1991-04-03 2017-06-07       2023-02-20         NaT   

   Gross_compensation  Plan_eligible_comp  \
0            82460.35            82460.35   
1            36328.55            36328.55   
2           319773.45           319773.45   
3            60120.66            60120.66   
4            71936.25            71936.25   

   Employee_deduction_1_Pre-tax_amount  Employee_deduction_2_Roth_Deferral  \
0                             17219.53                                 NaN   
1                              1018.17                                 NaN   
2                   

In [34]:
from datetime import datetime

today= pd.to_datetime('today')

df['Age'] = df['Birth_Date_MM/DD/YYYY'].apply(lambda x: (today.year - x.year) if pd.notnull(x) else None)
df['Ternure_Years'] = ((df['Termination_Date'].fillna(today)-df['Hire_Date']) / pd.Timedelta(days=365)).round(2)
df['Is_Active'] = df['Termination_Date'].isna()

print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   ID                                   82 non-null     object        
 1   Birth_Date_MM/DD/YYYY                82 non-null     datetime64[ns]
 2   Hire_Date                            82 non-null     datetime64[ns]
 3   Termination_Date                     19 non-null     datetime64[ns]
 4   Rehire_Date                          0 non-null      datetime64[ns]
 5   Gross_compensation                   82 non-null     float64       
 6   Plan_eligible_comp                   77 non-null     float64       
 7   Employee_deduction_1_Pre-tax_amount  59 non-null     float64       
 8   Employee_deduction_2_Roth_Deferral   1 non-null      float64       
 9   Employer_Match                       59 non-null     float64       
 10  Hours_worked_202

In [39]:
cutoff_date= pd.to_datetime('2023-01-01')

elegible_df = df[
    (df['Entry_date'] < cutoff_date) &
    (df['Is_Active']) &
    (df['Hours_worked_2022'] >= 1000) &
    (df['Employee_Status'].str.strip().str.lower() == 'full-time')
].copy()

print(f"Number of employees eligible for promotion: {len(elegible_df)}")

Number of employees eligible for promotion: 49


In [47]:
#Employee_Deferral_Pct = (Pre-tax  + Roth) / Plan_elegible_comp

elegible_df['Total_Deferral'] = elegible_df['Employee_deduction_1_Pre-tax_amount'].fillna(0) + \
                                 elegible_df['Employee_deduction_2_Roth_Deferral'].fillna(0)

elegible_df['Deferral_Pct'] = elegible_df['Total_Deferral'] / elegible_df['Plan_eligible_comp']

match_comp = elegible_df['Plan_eligible_comp'].clip(upper=305000)

elegible_df['Match_Pct'] = elegible_df['Deferral_Pct'].apply(lambda x: x if x <= 0.03 else (0.03 + min(x - 0.03, 0.02) * 0.5))

elegible_df['Match_Pct'] = elegible_df['Match_Pct'].clip(upper=0.04)

elegible_df['Expected_Employer_Match'] = (match_comp * elegible_df['Match_Pct']).round(2)

In [48]:
elegible_df['Employer_Match_Actual'] = elegible_df['Employer_Match'].fillna(0)
elegible_df['True_up_Amount'] = elegible_df['Expected_Employer_Match'] - elegible_df['Employer_Match_Actual']



In [50]:
display(elegible_df[['ID', 'Plan_eligible_comp', 'Total_Deferral', 'Deferral_Pct',
                     'Match_Pct', 'Expected_Employer_Match', 'Employer_Match_Actual', 'True_up_Amount']].head())

,ID,Plan_eligible_comp,Total_Deferral,Deferral_Pct,Match_Pct,Expected_Employer_Match,Employer_Match_Actual,True_up_Amount
0,Employee 1,82460.35,17219.53,0.208822,0.040000,3298.41,13948.47,-10650.06
2,Employee 3,319773.45,8716.71,0.027259,0.027259,8314.00,7472.55,841.45
3,Employee 4,60120.66,0.00,0.000000,0.000000,0.00,0.00,0.00
6,Employee 7,53881.01,0.00,0.000000,0.000000,0.00,0.00,0.00
9,Employee 10,53497.69,0.00,0.000000,0.000000,0.00,0.00,0.00


In [51]:
elegible_df['Expected_Employer_Match'] = elegible_df['Expected_Employer_Match'].clip(upper=12200)

elegible_df['True_up_Amount'] = (elegible_df['Expected_Employer_Match'] - elegible_df['Employer_Match'].fillna(0)).clip(lower=0).round(2)

In [52]:
def get_ineligibility_reason(row):
    reasons = []
    if pd.isna(row['Entry_date']) or row['Entry_date'] >= pd.Timestamp("2023-01-01"):
        reasons.append("Entry date after 2022")
    if not row['Is_Active']:
        reasons.append("Not active on 12/31/2022")
    if row['Hours_worked_2022'] < 1000:
        reasons.append("Less than 1000 hours")
    if str(row['Employee_Status']).strip().lower() != "full-time":
        reasons.append("Not full-time")
    return "; ".join(reasons) if reasons else None

# Crear df de no elegibles
non_eligible_df = df[~df.index.isin(elegible_df.index)].copy()
non_eligible_df['Ineligibility_Reason'] = non_eligible_df.apply(get_ineligibility_reason, axis=1)


In [53]:
# Elegibles con True-up
elegible_df.to_csv("eligible_true_up.csv", index=False)

# No elegibles con razones
non_eligible_df.to_csv("non_eligible_employees.csv", index=False)


In [64]:
def generar_resumen_true_up_por_id(employee_id, df):
    # Buscar el empleado
    row = df[df['ID'] == employee_id]

    if row.empty:
        print(f"Empleado con ID '{employee_id}' no encontrado en la lista de elegibles. \n Por favor revise el motivo de exclusión en el archivo non_eligible_employees.csv")
        return

    # Extraer fila
    row = row.iloc[0]

    # Construir resumen
    resumen = f"""
Resumen True-Up para el empleado: {employee_id}
------------------------------------------------------------
Frecuencia de pago: Mensual
Fórmula de match del empleador: 100% hasta 3%, 50% de 3%-5% (máx 4%)

Plan compensation anual: ${row['Plan_eligible_comp']:,.2f}
Total aportado por el empleado: ${row['Total_Deferral']:,.2f}
Porcentaje de ahorro anual: {row['Deferral_Pct']:.2%}
Porcentaje de match aplicado: {row['Match_Pct']:.2%}

Match del empleador durante el año: ${row['Employer_Match_Actual']:,.2f}
Match esperado al cierre del año: ${row['Expected_Employer_Match']:,.2f}
------------------------------------------------------------
True-up necesario (ajuste final): ${row['True_up_Amount']:,.2f}
"""
    print(resumen)



In [60]:
elegible_df.columns

Index(['ID', 'Birth_Date_MM/DD/YYYY', 'Hire_Date', 'Termination_Date',
       'Rehire_Date', 'Gross_compensation', 'Plan_eligible_comp',
       'Employee_deduction_1_Pre-tax_amount',
       'Employee_deduction_2_Roth_Deferral', 'Employer_Match',
       'Hours_worked_2022', 'Employee_Status', 'Entry_date', 'Age',
       'Ternure_Years', 'Is_Active', 'Total_Deferral', 'Deferral_Pct',
       'Match_Pct', 'Expected_Employer_Match', 'Employer_Match_Actual',
       'True_up_Amount'],
      dtype='object')

In [65]:
emp_id = input("Ingresa el ID del empleado: ")
generar_resumen_true_up_por_id(emp_id, elegible_df)


Ingresa el ID del empleado: Employee 2
Empleado con ID 'Employee 2' no encontrado en la lista de elegibles. 
 Por favor revise el motivo de exclusión en el archivo non_eligible_employees.csv
